In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import fast_oopsi
import constrained_oopsi
from scipy.io import loadmat, savemat

In [ ]:
totCell = loadmat('DataListCells.mat')

In [ ]:
totCell= loadmat('DataListCells.mat')
nCell  = 1
dff    = totCell['totCell'][nCell-1]['dff']
dff    = dff[0].astype('float64')[:,0]
spk    = totCell['totCell'][nCell-1]['spk']
spk    = spk[0]
caTime = totCell['totCell'][nCell-1]['CaTime']
caTime = caTime[0]
dt     = caTime[1] - caTime[0]

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
ax.plot(caTime, dff/dff.max())
ax.plot(spk, np.ones(len(spk))*1.1, 'ok')
ax.set_title('Calcium Fluorescence Signal')

In [ ]:
# wiener filter,
d, Cw, F_est, F_est_nonneg = fast_oopsi.wiener(dff.transpose(), dt=dt, iter_max=100)
fig, ax = plt.subplots()
ax.plot(caTime, dff/dff.max(), color='black', linewidth=1.0)
ax.plot(caTime, F_est_nonneg/dff.max(), color='red', linewidth=0.5)
ax.set_title('Reconstruct Calcium Fluorescence by wiener filter')

In [ ]:
# descritize,
d, v = fast_oopsi.discretize(dff/dff.max(), bins=[0.10])
fig, ax = plt.subplots()
ax.plot(d, color='red', linewidth=1.5)
ax.set_title('Reconstruct Spikes by discretized binning')

In [ ]:
# fast-oopsi,
d, Cz, P, F_est = fast_oopsi.fast(dff, dt=dt, iter_max=100)
fig, ax = plt.subplots()
ax.plot(caTime, dff/dff.max(), color='black', linewidth=1.0)
ax.plot(caTime, F_est/dff.max(), color='red', linewidth=0.5)
ax.set_title('Reconstruct Calcium Fluorescence by fast-oopsi')

In [ ]:
# constrained-fast-oopsi,
methods = ['cvxpy', 'spgl1', 'debug', 'cvx']
c, bl, c1, g, sn, spikes = constrained_oopsi.constrained_foopsi(dff, p=2, noise_range=[.25, .5], methods=methods[0])
fig, ax = plt.subplots()
ax.plot(caTime, dff/dff.max(), color='black', linewidth=1.0)
ax.plot(caTime, c/dff.max(), color='red', linewidth=0.5)
ax.set_title('Reconstruct Calcium Fluorescence by constrained oopsi filter')